In [1]:
import numpy as np
from load_mnist import MNIST

train_images, train_labels=MNIST(path="../Ehsan/Dataset/HW2/Q5",return_type="numpy",mode="vanilla").load_training()
test_images, test_labels=MNIST(path="../Ehsan/Dataset/HW2/Q5",return_type="numpy",mode="vanilla").load_testing()

for data in [test_images,train_images]:
    print("shape",data.shape)
    print("mean",data.mean())
    print("max",data.max())
    print("min",data.min())


shape (10000, 784)
mean 33.7912244898
max 255
min 0
shape (60000, 784)
mean 33.3184214498
max 255
min 0


In [3]:
def to_one_hat(y):
    o = np.zeros((len(y), 10))
    for i in range(len(y)):
        o[i][y[i] - 1] = 1
    return o

test_labels = to_one_hat(test_labels)
train_labels = to_one_hat(train_labels)

IndexError: arrays used as indices must be of integer (or boolean) type

In [4]:
def normalize(x):
    x = (x.T - np.mean(x, axis = 1)).T
    x = (x.T / np.std(x, axis = 1)).T
    return x

In [5]:
train_images = train_images.astype(float)
test_images = test_images.astype(float)

train_images = normalize(train_images)

test_images = normalize(test_images)         


In [9]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.model_selection import train_test_split
import time

## Define paramaters for the model
learning_rate = 0.05
n_epochs = 5
batch_size = 60
hidden_size = 100
learning_rate = 0.5

# regulation_rate = 1e-4

def normalize_batch(x , mu, ra_sigma):
    x = tf.transpose((tf.transpose(x) - tf.reduce_mean(x, axis = 1)))
    x = tf.transpose((tf.transpose(x) / tf.contrib.keras.backend.std(x, axis = 1)))
    return  tf.scalar_mul(ra_sigma, x) + mu
    
def normalizedFullLayer(input_data, output_size, act, std = 0.1):    
    ##defining the full linear Layer here
    w = tf.Variable(tf.random_normal([input_data.get_shape().as_list()[1], output_size], stddev = std))
    b = tf.Variable(tf.zeros([output_size]))
    a = tf.matmul(input_data, w) + b
    mu = tf.Variable(0.0, tf.float32)
    ra_sigma = tf.Variable(1.0, tf.float32)
    a = normalize_batch(a, mu, ra_sigma)
    return act(a), a

tf.reset_default_graph()

##define placeholder
## all image are 28 * 28 so x has 784 dimension
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
Y = tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')



##just some config for not getting whole server
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config = config)
for std in [0.1, 1, 10, 20]:
    ## the layers
    h, a1 = normalizedFullLayer(X, hidden_size, tf.nn.sigmoid, std)
    logits, _ = normalizedFullLayer(h, 10, tf.nn.sigmoid, std)

    ## defining loss function
    ## use cross entropy of softmax of logits as the loss function
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
    ## computes the mean over all the examples in the batch
    loss = tf.reduce_mean(entropy) 
    # + regulation_rate*tf.nn.l2_loss(w)  

    ##defining optimizer
    ## using gradient descent with learning rate of 0.5 to minimize loss
    gradient = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = gradient.minimize(loss)

    ##the prediction we made
    preds = tf.nn.softmax(logits)
    ##check how many of them are correct arg maxx is used because Y is one hat
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

    ## to visualize using TensorBoard
    #writer = tf.summary.FileWriter('./graphs/mnist/c', sess.graph)
    ####cross validation
    ##starting time
    start_time = time.time()
    ##initialize the variables
    sess.run(tf.global_variables_initializer())
   
    n_batches = int(60000 / batch_size)
    for i in range(n_epochs):  # train the model n_epochs times
        total_loss = 0
        total_acc = 0

        conc = list(zip(train_images, train_labels))
        random.shuffle(conc)
        train_images, train_labels = zip(*conc)

        for j in range(n_batches):
            X_batch, Y_batch = train_images[j* batch_size:(j+1)*batch_size], train_labels[j* batch_size:(j+1)*batch_size]
            ##training batches
            _, loss_batch, acc_batch, test = sess.run([optimizer, loss, accuracy, a1], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += loss_batch
            total_acc += acc_batch
            ## they said that show every 20 step but that would be too much, so i show every 200 step
            if j % 200 == 199:
                print('step {}, with std_dev {}, Average loss : {}, Accuracy : {:.6f}'.format(i * n_batches + j, std, total_loss / 200, total_acc / batch_size / 200))
#                 print(np.mean(np.absolute(test)))
                total_loss = 0
                total_acc = 0

    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')

    # test the model
    ##number of test batches
    n_batches = int(10000 / batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        ##test batches
        X_batch, Y_batch = test_images[i* batch_size:(i+1)*batch_size], test_labels[i* batch_size:(i+1)*batch_size]
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch[0]

    print('Accuracy {}, with std_dev {}'.format(total_correct_preds / 10000, std))

sess.close()
'''
در سوال 5.1 همانطور که میبینید بعد از نرمالایز کردن ورودی اکیوریسی هم تست هم ترین از حدود 82 به 88 می رسد
در قسمت 5.2 همانطور که در ریزالت ها  میتوانید ببینید با انحراف معیار مناسب با بچ نرمالیزیشن به اکیوریسی 96 درصد هم شبکه میرسد
نتیجه ای که گرفتیم این بود با زیاد شدن لرنینگ ریت شبکه ی چند لایه همانطور که انتظار میرفت اکیوریسی اش خیلی پایین است زیرا وقتی لرنینگ ریت که خیلی زیاد است دور لاس این ور آن ور میرود
ولی همانطور که در سوال های تطوری دیده بودیم شبکه ی با بچ نورمالیزیشن نسبت به لرنینگ ریت طیاد حساس نیست و میتوان لرنینگ ریت زیاد داد
اما جفت شیکه وقتی انحراف معیار وزن ها زیاد می شود مقدار اولیه اندازه خود وزن ها زیاد می شود و اکسپلود گرادیان رخ می دهد 
'''

step 199, with std_dev 0.1, Average loss : 1.7617998731136322, Accuracy : 0.744333
step 399, with std_dev 0.1, Average loss : 1.5854084551334382, Accuracy : 0.892000
step 599, with std_dev 0.1, Average loss : 1.5573463553190232, Accuracy : 0.910167
step 799, with std_dev 0.1, Average loss : 1.5394835656881332, Accuracy : 0.924667
step 999, with std_dev 0.1, Average loss : 1.52846699655056, Accuracy : 0.933417
step 1199, with std_dev 0.1, Average loss : 1.5204041415452958, Accuracy : 0.941000
step 1399, with std_dev 0.1, Average loss : 1.5152849197387694, Accuracy : 0.945000
step 1599, with std_dev 0.1, Average loss : 1.5104277181625365, Accuracy : 0.948917
step 1799, with std_dev 0.1, Average loss : 1.5079844695329667, Accuracy : 0.951083
step 1999, with std_dev 0.1, Average loss : 1.5050714641809464, Accuracy : 0.954917
step 2199, with std_dev 0.1, Average loss : 1.5003749334812164, Accuracy : 0.958667
step 2399, with std_dev 0.1, Average loss : 1.4986405688524247, Accuracy : 0.961333